# Studies on Softening and Hardening

This notebook illustrates researches presented in the following papers. It is best to read one of these articles before running this notebook.
- Jean-Paul Delahaye et Philippe Mathieu. Méta-Stratégies pour le Dilemme Itéré du Prisonnier. [JFSMA 2016, Cepadues eds](https://www.cepadues.com/livres/sciences/donnees-informatique-ia-ihm/1099-jfsma-2016-systemes-multi-agents-et-simulation-9782364935594.html), ISBN 978-2-36493-559-4. pp 13-22 [(download)](https://hal.inria.fr/hal-01378567/document)
- Jean-Paul Delahaye et Philippe Mathieu. Adoucir son comportement ou le durcir. 23 mars 2016, [POUR LA SCIENCE n 462](https://www.pourlascience.fr/sd/cosmologie/pour-la-science-n0462-750.php). pp 80-85 [(download)](https://www.cristal.univ-lille.fr/~jdelahay/pls/2016/269.pdf)
- Softening and Hardening in the Iterated Prisoner’s Dilemma. [System Man and Cybernetics](https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=6221021) - Systems. SMC 2022 (ToAppear)


We present here two methods to evaluate and differentiate the general behaviors of cooperation and aggressiveness of strategies in the iterated prisoner’s dilemma (IPD). 
- The first method involves taking classes of strategies, grouping strategies into subclasses based on their cooperative or aggressive “temperament” and then comprehensively assessing these subclasses. Four kinds of behaviors are confronted and compared. 
- The second method is to operate transformations that “soften” or “harden” strategies of a given set and compare them with the results obtained with the initial set. 

What we establish conforms to the classical analyses of the IPD. But what it shows is more precise and subtle and furthermore is established with new methods that can be considered as experimental evidence. Our method for conducting comparisons between general strategies behaviors is a new tool that can have multiple applications regarding the prisoner’s dilemma of other games.

## Init and recalls

In [ ]:
# Load the classical modules for IPD
# Game about game theory, ipd about Iterated prisoner's dilemma, Strategies for a set of strategies

%run ../src/game.py
%run ../src/ipd.py
%run ../src/strategies.py

g.prettyPrint()   # prisoner's dilemma

In [ ]:
# example of meeting
m = Meeting(g, Periodic('ccd'), Tft() , length=10)
m.run()
m.prettyPrint()

In [ ]:
# Most strategies are generic and have specific constructors. See strategies.py
# Some available strategies
bag = getClassicals()  # getMem(1,1)
for s in bag :
    print(s.name , end=', ')

In [ ]:
# example of a classical tournament
bag = [Periodic('C','allC'), Periodic('D','allD'), Tft(), Periodic('CCD','perCCD')]
t = Tournament(g,bag,length=10)
t.run()
t.matrix

In [ ]:
e = Ecological(t,pop=100, max_iter=1000)
e.run()
e.drawPlot()

## Classification of strategies

When we look at a set of strategies we see that they have general characteristics that allow them to be classified into large families. The four families we consider are


- IC. a strategy is “initially cooperative” (IC) if it never takes the initiative of defection
- SA. we call “spontaneously aggressive” the complementary set to IC
- IA. a strategy is “initially aggressive” if it never takes the initiative of cooperation
- SC. we call “spontaneously cooperative” the complementary set to IA

Note that each IA strategy is also SA and that each IC strategy is also SC.
On the other hand, it is possible to be IC and SC at the same time.

The `getAgressivityClass` method takes a set of strategies as a parameter and breaks these strategies down into a 4 entry dictionary; one for each of these classes

In [ ]:
def getAgressivityClasses(bag):
    aggressivity = {'IC':[], 'SC':[],'IA':[],'SA':[]}
    for strat in bag : 
        m1 = Meeting(g, strat, Periodic('C'), 100)
        m1.run()
        m2 = Meeting(g, strat, Periodic('D'), 100)
        m2.run()
        if m1.s1_score == 300:
             aggressivity['IC'] += [strat]
        if m1.s1_score > 300:
             aggressivity['SA'] += [strat]
        if m2.s1_score == 100:
             aggressivity['IA'] += [strat]
        if m2.s1_score < 100:
             aggressivity['SC'] += [strat]
    return aggressivity
        

ac = getAgressivityClasses(getMem11()) #getClassicals()[0:17])
for cle,valeur in ac.items():
    print (cle, len(valeur) , [s.name for s in valeur])

# Take care that the 17th is SpitefulCC, contrary to the Simpat paper in which 
# it's mem2s

## List of strategies in a family, with their categories

In [ ]:
import pandas as pd
bag = getClassicals()[0:17]
ac = getAgressivityClasses(bag)
result=[]
for s in bag :
    lig=[
    'IA' if s in ac['IA'] else '' ,
    'IC' if s in ac['IC'] else '' ,
    'SA' if s in ac['SA'] else '' ,
    'SC' if s in ac['SC'] else ''
    ]
    result.append(lig)
df = pd.DataFrame(result, index=[s.name for s in bag])
df

## Size of each class for each of the main families

In [ ]:
import pandas as pd
result=[]
for bag in [getClassicals()[0:17],getMem(1,1),getMem(1,2),getAllProba(5)] :
    ac = getAgressivityClasses(bag)
    lig=[]
    for key,value in ac.items():
        lig.append(len(value))
    result.append(lig)
df = pd.DataFrame(result, columns=list(ac.keys()), index=['classicals','mem11','mem12','mem21'])
df



Aggressiveness of strategies before and after simplification

In [ ]:
# import tools to simplify a set of strategies
%run ../src/tools.py

def createTab(bag):
    print('Initially :\t',len(bag))
    agr = getAgressivityClasses(bag)
    simplified = simplifyWithTournament(bag, [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
    print('After simplification :\t',len(simplified))
    agrS = getAgressivityClasses(simplified)
    tab = pd.DataFrame(
            np.nan, ["IC","SC","IA", "SA"], ["Before simplify","After simplify"]
        )
    for key in agr :
        tab.at[key,"Before simplify" ] = len(agr[key])
    for key in agrS :
        tab.at[key, "After simplify"] = len(agrS[key])
    pd.options.display.float_format = '{:,.0f}'.format
    return tab

createTab(getMem(1,1))


# Performance of the strategies in each class?

In [ ]:
bag = getMem(1,1) # getClassicals()[0:17]
t = Tournament(g,bag,100)
e = Ecological(t, 100)
e.run()
ranking = e.historic.iloc[e.generation].rank(0, method="min", ascending=False)
score = e.historic.iloc[e.generation]
agr = getAgressivityClasses(bag)

# Generate the data with mean of ranks and scores
tab = pd.DataFrame(
            np.nan, ["IC","SC","IA", "SA"], ["Mean of ranks","Mean of scores"]
        )
for key in agr:
    ranks = []
    scores = []
    for strat in agr[key]:
        ranks += [ranking[strat.name]]
        scores += [score[strat.name]]

    tab.at[key, "Mean of ranks"] = np.mean(ranks)
    tab.at[key, "Mean of scores"] = np.mean(scores)
pd.options.display.float_format = '{:,.0f}'.format   
print(tab)

# Generate the graph with mean of scores 
ranksIC, ranksSC, ranksIA, ranksSA = [], [], [], []
for i in range(e.generation):
    rIC = 0
    rSC = 0
    rIA = 0
    rSA = 0
    for key in agr:
        for strat in agr[key]:
            if key == "IC":
                rIC += e.historic.iloc[i][strat.name]
            if key == "SC":
                rSC += e.historic.iloc[i][strat.name]
            if key == "IA":
                rIA += e.historic.iloc[i][strat.name]
            if key == "SA":
                rSA += e.historic.iloc[i][strat.name]
    ranksIC += [np.sum(rIC)/len(agr['IC'])]
    ranksSC += [np.sum(rSC)/len(agr['SC'])]
    ranksIA += [np.sum(rIA)/len(agr['IA'])]
    ranksSA += [np.sum(rSA)/len(agr['SA'])]


fig, ax = plt.subplots()
ax.plot(ranksIC, label='IC')
ax.plot(ranksSC, label= 'SC')
ax.plot(ranksSA, label='SA')
ax.plot(ranksIA, label='IA')
ax.set_ylabel('Scores')
ax.set_xlabel('Generation')
ax.set_facecolor('#F0F0F0')
ax.grid()
ax.legend(loc='best')
# fig.savefig('toto.png', dpi=500)
plt.show()
 

# Transformation methods

In [ ]:
# Soften : less than k defections since the beginning then I Cooperate, else S
class Method1 (Strategy):
    def __init__(self,type,k,strat):
        self.type=type.upper()
        self.k = k
        self.strat = strat
        self.hasDefected=0
        self.hasCooperated=0
        self.name=self.type+'1_K'+str(k)+'_'+self.strat.name
        
    def getAction(self,tick):
        c = self.strat.getAction(tick)

        if self.type=='SOFTEN' and self.hasDefected<self.k :
            #print("softer at",self.k)          
            return 'C';
        if self.type=='HARDEN' and self.hasCooperated<self.k :
            #print("harder at",self.k)
            return 'D';
            
        return c;
    
    def clone(self):
        object = Method1(self.type,self.k, self.strat.clone())
        return object
    
    def update(self,my,his):
        if his=='D' : self.hasDefected+=1
        if his=='C' : self.hasCooperated+=1
        self.strat.update(my,his)


In [ ]:
#   TESTS

# Softening AllD        
s1 = Method1('soften',2, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Softening AllD        
s1 = Method1('soften',3, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method1('harden',2, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method1('harden',3, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()

In [ ]:
# Soften : OR operator :  one Cooperation during the k last rounds then I Cooperate, else S
class Method2 (Strategy):
    def __init__(self,type,k,strat):
        self.type=type.upper()
        self.k = k
        self.strat = strat
        self.hasDefected=0
        self.hasCooperated=0
        self.name=self.type+'2_K'+str(k)+'_'+self.strat.name
        self.his=''
        
    def getAction(self,tick):
        c = self.strat.getAction(tick)

        if (tick > 0) : #self.k - 1) :
            if self.type=='SOFTEN' and 'C' in self.his[-self.k:] :
                # print("softer at",self.k)
                return 'C';
            if self.type=='HARDEN' and 'D' in self.his[-self.k:] :
                # print("harder at",self.k)
                return 'D';
            
        return c;
    
    def clone(self):
        object = Method2(self.type,self.k, self.strat.clone())
        return object
    
    def update(self,my,his):
        self.his+=his
        self.strat.update(my,his)


In [ ]:
#    TESTS

# Softening AllD        
s1 = Method2('soften',2, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Softening AllD        
s1 = Method2('soften',3, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method2('harden',2, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method2('harden',3, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()


In [ ]:
# Soften : AND operator :  k Cooperations during the k last rounds then I Cooperate, else S

class Method3 (Strategy):
    def __init__(self,type,k,strat):
        self.type=type.upper()
        self.k = k
        self.strat = strat
        self.name=self.type+'3_K'+str(k)+'_'+self.strat.name
        self.his=''
        
    def getAction(self,tick):
        c = self.strat.getAction(tick)
        if (tick > self.k - 1) :
            if (self.type=='SOFTEN' and self.his[-self.k : ]=='C'*self.k) :
                # Il a coopere k fois consecutives
                # print("softer at",self.k)
                return 'C';
            if (self.type=='HARDEN' and self.his[-self.k : ]=='D'*self.k) :
                # Il a trahi k fois consecutives
                # print("harder at",self.k)
                return 'D';
            
        return c;
    
    def clone(self):
        object = Method3(self.type,self.k, self.strat.clone())
        return object
    
    def update(self,my,his):
        self.his+=his
        self.strat.update(my,his)

In [ ]:
#   TEST   

# Softening AllD        
s1 = Method3('soften',2, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Softening AllD        
s1 = Method3('soften',3, Periodic('D'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method3('harden',2, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()
print()

# Hardening AllC   
s1 = Method3('harden',3, Periodic('C'))
s2 = Periodic('CCD')
m=Meeting(g,s1,s2,20)
m.run()
m.prettyPrint()

Note that with k=1, Method2 and Method3 are equivalent

# Competition between all variations on a given bag

In [ ]:
%%time
bag=getClassicals()[0:17] # getMem(1,1)
bagM1SoftenK1=[Method1("soften",1,s) for s in bag]
bagM1HardenK1=[Method1("harden",1,s) for s in bag]
bagM2SoftenK1=[Method2("soften",1,s) for s in bag]
bagM2HardenK1=[Method2("harden",1,s) for s in bag]
bagM3SoftenK1=[Method3("soften",1,s) for s in bag]
bagM3HardenK1=[Method3("harden",1,s) for s in bag]
# To help grouping, we add 'Normal____' (10 chars) as prefix 
for s in bag:
    s.name='Normal____'+s.name
    
allbags=bag+bagM1SoftenK1+bagM1HardenK1+bagM2SoftenK1+bagM2HardenK1 # +bagM3SoftenK1+bagM3HardenK1

t = Tournament(g,allbags,100)
t.run()
e = Ecological(t,100)
e.run()
# e.drawPlot()

## Then we group the results by family

We add to the 17 initial strategies, their versions softened and hardened by the 4 corresponding transformations, which gives 85 strategies, then we cumulate the results of each of the 5 classes (1+4) which allows to classify them.

see Fig4, part A , SMC2022

One can see that softening improve the classical strategies. 

The ranking is here SOFTEN1 > SOFTEN2 > NORMAL > HARDEN2 > HARDEN1   

In [ ]:
x=e.historic.copy()
new_columns=[n[0:10] for n in x.columns]
x.columns=new_columns
result = x.groupby(by=x.columns , axis=1).sum()
result.plot()
plt.grid()
# plt.savefig('fig4_smc.png', dpi=500)

#x.groupby(by=x.columns , axis=1).agg({'mean':[np.mean] , 'std':[np.std]})

In [ ]:
# Nevertheless, sometimes hardening also improves a classic strategy

# Try the same experience with getMem(1,1)

# Variations with "Improved AllD"

see Table5 smc2021


In [ ]:
for k in range(0,10):
    # Competition with SOFTEN_allD
    bag=getMem(1,1) + getClassicals()[0:17] # + getAllProba(1)
    # print(len(bag), 'stategies involved')
    bag = bag + [ Method1('soften',k, Periodic('D','allD')) ]
    t = Tournament(g,bag,100)
    t.run()
    e = Ecological(t,100)
    e.run()
    # rank of SOFTEN_allD
    df=pd.DataFrame(e.historic[-1:].transpose()) # .sort_values(by='m2',ascending=False)
    df.columns=['score']
    df['rank'] = df['score'].rank(axis=0, ascending=False, method='min')  # average,min,max,first,dense
    df
    print(k,df.loc['SOFTEN1_K'+str(k)+'_allD']['rank'])
    

see Fig6 SMC2022 

In [ ]:
# If length=1000 the 3 soften are equal. If length=100 soften1_k2 is the lead

bag=getMem(1,1) # getClassicals()[0:17] # 
bag = bag + [ Method1('soften',2, Periodic('D','allD')) ]
bag = bag + [ Method1('soften',3, Periodic('D','allD')) ]
bag = bag + [ Method1('soften',4, Periodic('D','allD')) ]

t = Tournament(g,bag,1000)
e = Ecological(t,100)
e.run()
e.drawPlot(nbLegends=10) #, file='fig6_allD_soften_Classicals.png')
# e.historic